In [10]:
import requests
import pandas as pd

response = requests.get("http://api.citybik.es/v2/networks")
data = response.json()

city_name = input("Enter name of city: ")
filtered_networks = []
for network in data["networks"]:
    if network["location"]["city"].lower() == city_name.lower():
        filtered_networks.append(network)

if filtered_networks:
    chosen_network = filtered_networks[0]
    chosen_network_id = chosen_network["id"]

    chosen_network_response = requests.get(f"http://api.citybik.es/v2/networks/{chosen_network_id}")
    
    chosen_network_data = chosen_network_response.json()

    bike_stations = chosen_network_data["network"]["stations"]
    
    station_data = []
    for station in bike_stations:
        station_name = station["name"]
        latitude = station["latitude"]
        longitude = station["longitude"]
        total_bikes = station["free_bikes"] + station["empty_slots"]
        
        station_data.append({
            "station_name": station_name,
            "Latitude": latitude,
            "Longitude": longitude,
            "Total Bikes": total_bikes
        })

    if station_data:
        df = pd.DataFrame(station_data)
        print(f"Bike Stations in {city_name} (DataFrame):\n")
        print(df)
    else:
        print("No bike station data available.")
else:
    print(f"No networks found for {city_name}.")
    




Bike Stations in paris (DataFrame):

                            station_name   Latitude  Longitude  Total Bikes
0          Benjamin Godard - Victor Hugo  48.865983   2.275725           32
1     André Mazet - Saint-André des Arts  48.853756   2.339096           54
2        Faubourg Du Temple - Republique  48.867872   2.364898           35
3                 Beaux-Arts - Bonaparte  48.856452   2.334852           19
4                     Toudouze - Clauzel  48.879296   2.337360           21
...                                  ...        ...        ...          ...
1451      Henri Barbusse - Marcel Cachin  48.937421   2.372037           37
1452        Stéphanie Bouvard - Gambetta  48.814548   2.402439           30
1453                              Mairie  48.879487   2.416153           20
1454          Rouget de L'isle - Watteau  48.778193   2.396302           20
1455              Stade Gaston Bouillant  48.939959   2.327957           30

[1456 rows x 4 columns]


In [11]:
# view extracted dataframe
print(df.head(3))

                         station_name   Latitude  Longitude  Total Bikes
0       Benjamin Godard - Victor Hugo  48.865983   2.275725           32
1  André Mazet - Saint-André des Arts  48.853756   2.339096           54
2     Faubourg Du Temple - Republique  48.867872   2.364898           35


##### Next we need to combine latitude and longitude into a string separated by ',' so that we can use it to retrieve locations from foursquare API.

In [22]:
# Create a new column concatenating latitude and longitude into a comma-separated string
df['ll'] = df['Latitude'].astype(str) + ',' + df['Longitude'].astype(str)

# Use station_name as index column
df.set_index('station_name', inplace=True)

print(df)





                                     Latitude  Longitude  Total Bikes  \
station_name                                                            
Benjamin Godard - Victor Hugo       48.865983   2.275725           32   
André Mazet - Saint-André des Arts  48.853756   2.339096           54   
Faubourg Du Temple - Republique     48.867872   2.364898           35   
Beaux-Arts - Bonaparte              48.856452   2.334852           19   
Toudouze - Clauzel                  48.879296   2.337360           21   
...                                       ...        ...          ...   
Henri Barbusse - Marcel Cachin      48.937421   2.372037           37   
Stéphanie Bouvard - Gambetta        48.814548   2.402439           30   
Mairie                              48.879487   2.416153           20   
Rouget de L'isle - Watteau          48.778193   2.396302           20   
Stade Gaston Bouillant              48.939959   2.327957           30   

                                                  